# Filled Area Plot

## Basic Filled Area Plot

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
# Sample data
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
values = [10, 13, 11, 15, 12, 16]

# Create DataFrame
df = pd.DataFrame({
    'Month': months,
    'Value': values
})

# Create basic filled area plot
fig = px.area(
    df,
    x='Month',
    y='Value',
    title='Basic Filled Area Plot',
    labels={'Value': 'Amount', 'Month': 'Month'}
)

# Update layout
fig.update_layout(
    showlegend=False,
    hovermode='x unified'  # Shows all data points at the same x-coordinate in a single tooltip
)

fig.show()

## Multiple lines with filled area between them

- Days with the highest and lowest temperatures
- Days with the largest temperature fluctuations
- Overall temperature trends throughout the week


In [5]:
# Sample data
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
min_temp = [68, 65, 70, 72, 69, 73, 71]
max_temp = [82, 79, 84, 89, 85, 90, 86]

# Create a DataFrame for easier data manipulation
df = pd.DataFrame({
    'day': days,
    'min_temp': min_temp,
    'max_temp': max_temp
})
print(df)

# Melt the dataframe to long format
# The melt() function reshapes the DataFrame from wide to long format:
# - id_vars: Column(s) to keep as identifier variables (not reshaped)
# - value_vars: Column(s) to "unpivot" from columns to rows
# - var_name: Name for the new column containing the former column names
# - value_name: Name for the new column containing the values
# This transforms the data from having separate min_temp and max_temp columns to having a single temperature_type column (with values 'min_temp' or 'max_temp') and a temperature column containing all the values
df_melted = df.melt(id_vars='day', value_vars=['min_temp', 'max_temp'], var_name='temperature_type', value_name='temperature')
df_melted

         day  min_temp  max_temp
0     Monday        68        82
1    Tuesday        65        79
2  Wednesday        70        84
3   Thursday        72        89
4     Friday        69        85
5   Saturday        73        90
6     Sunday        71        86


,day,temperature_type,temperature
0,Monday,min_temp,68
1,Tuesday,min_temp,65
2,Wednesday,min_temp,70
3,Thursday,min_temp,72
4,Friday,min_temp,69
5,Saturday,min_temp,73
6,Sunday,min_temp,71
7,Monday,max_temp,82
8,Tuesday,max_temp,79
9,Wednesday,max_temp,84


In [6]:
# Create the figure using plotly express
fig = px.line(
    df_melted,
    x='day',
    y='temperature',
    color='temperature_type',
    labels={'temperature': 'Temperature (°F)', 'day': 'Day of Week'},
    title='Weekly Temperature Range',
    color_discrete_map={'min_temp': 'rgb(31, 119, 180)', 'max_temp': 'rgb(255, 127, 14)'}
)

# Fill between the lines
fig.update_traces(fill='tonexty',
                  selector=dict(name='max_temp'))

# Update layout for hovermode
fig.update_layout(hovermode='x unified')

# Show the figure
fig.show()

## Filled Area Plot with Customization Calulation

- The filled area plot shows the stock price over time, with the area between the moving average and the stock price shaded to indicate volatility.
- The moving average line smooths out the stock price fluctuations, making it easier to identify trends.
- The shaded area between the moving average and the stock price represents the volatility of the stock, with wider bands indicating higher volatility.
- The example also includes hover information to display the stock price and moving average values when hovering over the plot.

In [7]:
import numpy as np

# Generate simulated stock price data for demonstration
# Using a random seed ensures we get the same "random" values each time
np.random.seed(42)
dates = pd.date_range(start='2024-01-01', periods=30)  
price = 100 + np.cumsum(np.random.normal(0, 1, 30))    
ma_7 = pd.Series(price).rolling(window=7).mean()       
volatility = pd.Series(price).rolling(window=5).std() * 2 

# STEP 2: Organize data into a structured DataFrame
df = pd.DataFrame({
    'Date': dates,
    'Stock Price': price,
    '7-Day MA': ma_7,                 # Moving average line
    'Upper Band': ma_7 + volatility,  # Upper volatility band
    'Lower Band': ma_7 - volatility   # Lower volatility band
})

# STEP 3: Reshape data for Plotly Express using melt
# This transforms the data from wide to long format, which works better with px
df_melted = df.melt(id_vars=['Date'], 
                    value_vars=['Stock Price', '7-Day MA', 'Upper Band', 'Lower Band'],
                    var_name='Series', value_name='Value')
df_melted

,Date,Series,Value
0,2024-01-01,Stock Price,100.496714
1,2024-01-02,Stock Price,100.358450
2,2024-01-03,Stock Price,101.006138
3,2024-01-04,Stock Price,102.529168
4,2024-01-05,Stock Price,102.295015
...,...,...,...
115,2024-01-26,Lower Band,95.032425
116,2024-01-27,Lower Band,94.531125
117,2024-01-28,Lower Band,95.045873
118,2024-01-29,Lower Band,94.637671


In [8]:
# STEP 4: Create the base line chart with all series
fig = px.line(df_melted, 
              x='Date', 
              y='Value', 
              color='Series',
              title='Stock Price with Moving Average and Volatility Bands',
              labels={'Value':'Price ($)'})

# STEP 5: Add the volatility bands as separate traces
# We need to add these again to properly set up the fill between them
fig.add_trace(px.line(df, x='Date', y='Upper Band').data[0])
fig.add_trace(px.line(df, x='Date', y='Lower Band').data[0])

# STEP 6: Customize the appearance of each trace
# Create fill between upper and lower volatility bands
fig.update_traces(fill='tonexty', selector=dict(name="Lower Band"), 
                 fillcolor='rgba(255, 165, 0, 0.2)')  # Light orange shading for volatility

# Style the main stock price line
fig.update_traces(line=dict(color='darkgreen', width=3), 
                 selector=dict(name="Stock Price"))

# Style the moving average line
fig.update_traces(line=dict(color='orange', width=2, dash='dash'), 
                 selector=dict(name="7-Day MA"))

# Hide the lines for volatility bands (we only want the filled area)
fig.update_traces(line=dict(width=0), selector=dict(name="Upper Band"))
fig.update_traces(line=dict(width=0), selector=dict(name="Lower Band"))

# STEP 7: Final layout adjustments
fig.update_layout(
    hovermode='x unified'  # Show all values when hovering on a specific date
)

# Display the completed chart
fig.show()